# Day 21, expressive symians

Today's puzzle forms an acyclic directed graph of expressions. Simply construct the graph and compute the expression!

In [1]:
import operator
from dataclasses import dataclass
from typing import Callable, Final


OPERATORS: Final[dict[str, Callable[[int, int], int]]] = {
    "+": operator.add,
    "*": operator.mul,
    "-": operator.sub,
    "/": operator.floordiv,
}


@dataclass
class Monkey:
    name: str

    def __call__(self, expr: "MonkeyExpression") -> int:
        raise NotImplementedError

    @classmethod
    def from_string(cls, string: str) -> "Monkey":
        name, *parts = string.split()
        name = name[:-1]
        match parts:
            case [value]:
                return ValueMonkey(name, int(value))
            case [left, operator, right] if operator in OPERATORS:
                return ExpressionMonkey(name, left, right, OPERATORS[operator])
            case _:
                raise ValueError("Invalid monkey expression")


@dataclass
class ExpressionMonkey(Monkey):
    left: str
    right: str
    operator: Callable[[int, int], int]

    def __call__(self, monkeys: dict[str, Monkey]) -> int:
        left, right = monkeys[self.left], monkeys[self.right]
        return self.operator(left(monkeys), right(monkeys))


@dataclass
class ValueMonkey(Monkey):
    value: int

    def __call__(self, *_) -> int:
        return self.value


@dataclass
class MonkeyExpression:
    monkeys: dict[str, Monkey]

    @classmethod
    def from_text(cls, text: str) -> "MonkeyExpression":
        return cls(
            {(m := Monkey.from_string(line)).name: m for line in text.splitlines()}
        )

    def __getitem__(self, name: str) -> Monkey:
        return self.monkeys[name]

    def __call__(self) -> int:
        return self.monkeys["root"](self.monkeys)


example = MonkeyExpression.from_text(
    """\
root: pppw + sjmn
dbpl: 5
cczh: sllz + lgvd
zczc: 2
ptdq: humn - dvpt
dvpt: 3
lfqf: 4
humn: 5
ljgn: 2
sjmn: drzm * dbpl
sllz: 4
pppw: cczh / lfqf
lgvd: ljgn * ptdq
drzm: hmdt - zczc
hmdt: 32
"""
)
assert example() == 152

In [2]:
import aocd


expression = MonkeyExpression.from_text(aocd.get_data(day=21, year=2022))
print("Part 1:", expression())

Part 1: 353837700405464


## Part 2: solving the monkey puzzle tree

Suddenly, the tree turns out to be an equality equation. So lets turn it into one! We've used the [`sympy` symbolic math library](https://www.sympy.org/en/), specifically, for [2019, Day 2 (part 2)](../2019/Day%2002.ipynb) and for [2021, Day 24](../2021/Day%2024.ipynb).

Lets use that again here. We want to [solve an equation numerically](https://docs.sympy.org/latest/modules/solvers/solvers.html#sympy.solvers.solvers.nsolve), that is, give sympy our series of monkeys (symbols) and have it produce the value for `humn`.

We can't use floor division (`//`) with sympy, luckily the puzzle equations work just fine with true division (`/`) and rounding.

In [3]:
import sympy as sy


def solve_for_human(expression: MonkeyExpression) -> int:
    monkeys = expression.monkeys.copy()
    for name, monkey in monkeys.items():
        if isinstance(monkey, ExpressionMonkey) and monkey.operator is operator.floordiv:
            monkeys[name] = ExpressionMonkey(name, monkey.left, monkey.right, operator.truediv)
    humn = sy.Symbol("humn")
    monkeys["humn"] = lambda *_: humn
    root = monkeys["root"]
    left, right = monkeys[root.left](monkeys), monkeys[root.right](monkeys)
    [result] = sy.solveset(left - right)
    return round(result)


assert solve_for_human(example) == 301

In [4]:
print("Part 2:", solve_for_human(expression))

Part 2: 3678125408017
